In [1]:
import pandas as pd
import numpy as np

In [2]:
medal_standings_path = r"C:\Users\U6079006\Desktop\PowerBI Projects\Onyx Project\September\medal_standings.xlsx"
medal_athlete_path = r"C:\Users\U6079006\Desktop\PowerBI Projects\Onyx Project\September\medal_athlete.xlsx"

# Read the data from both Excel files
medal_standings_df = pd.read_excel(medal_standings_path, sheet_name="Data")  # Load all sheets to inspect
medal_athlete_df = pd.read_excel(medal_athlete_path, sheet_name="Data")      # Load all sheets to inspect

In [3]:
print(medal_standings_df.shape)
print(medal_athlete_df.shape)

(2346, 18)
(29170, 27)


### Function to clean and preprocess the data

In [4]:
# 1. Handling Missing Values
# Fill missing sports with "Overall" if rank_type is "Overall", otherwise retain as NaN
medal_standings_df['sport'].fillna('Overall', inplace=True)
 
# Fill missing numeric columns with 0 where it makes sense
numeric_cols = ['npc_gold', 'npc_silver', 'npc_bronze']
medal_standings_df[numeric_cols] = medal_standings_df[numeric_cols].fillna(0)
 
    
    
# 2. Standardizing Text Formats
# Standardize country and athlete names by removing extra spaces and fixing capitalization
medal_standings_df['npc_name'] = medal_standings_df['npc_name'].str.strip().str.title()
medal_athlete_df['npc_name'] = medal_athlete_df['npc_name'].str.strip().str.title()
medal_athlete_df['athlete_name'] = medal_athlete_df['athlete_name'].str.strip().str.title()



# Overall Sport_code
medal_standings_df['sport_code'].replace('', 'Overall', inplace=True)
medal_standings_df['sport_code'].fillna('Overall', inplace=True)


# Remove Duplicates
medal_athlete_df = medal_athlete_df.drop_duplicates()
medal_standings_df = medal_standings_df.drop_duplicates()
 
# Convert 'medal' column to title case (Gold, Silver, Bronze)
medal_athlete_df['medal'] = medal_athlete_df['medal'].str.title()
 
# 3. Datetime Conversion
# Convert 'games_start' and 'games_end' to datetime format
medal_standings_df['games_start'] = pd.to_datetime(medal_standings_df['games_start'] + ' ' + medal_standings_df['games_year'].astype(str), errors='coerce')
medal_standings_df['games_end'] = pd.to_datetime(medal_standings_df['games_end'] + ' ' + medal_standings_df['games_year'].astype(str), errors='coerce')
 
    
# 4. Feature Engineering
# Calculate total medals
medal_standings_df['npc_total_medals'] = medal_standings_df[['npc_gold', 'npc_silver', 'npc_bronze']].sum(axis=1)
 
# Calculate medals per athlete
medal_athlete_df['medals_per_athlete'] = medal_athlete_df.groupby('athlete_name')['medal'].transform('count')
 
# Calculate games duration in days
medal_standings_df['games_duration'] = (medal_standings_df['games_end'] - medal_standings_df['games_start']).dt.days
 
# Medal density (medals per day of games) for each country
medal_standings_df['medal_density'] = medal_standings_df['npc_total_medals'] / medal_standings_df['games_duration']
 
# Flag if a country is top-performing (more than 10 medals)
medal_standings_df['is_top_performer'] = medal_standings_df['npc_total_medals'] > 10
 
    
    
# 5. Data Enrichment
# Map continents to countries for consistency, handle exceptions
continent_map = {
    "Europe[a]": "Europe",
    "Oceania": "Oceania",
    "Americas": "Americas",
    "Asia": "Asia",
    "Africa": "Africa"
}
medal_standings_df['games_continent'] = medal_standings_df['games_continent'].map(continent_map)
 

# 6. Additional Calculations
# Create a binary column to indicate whether the country is the host
medal_standings_df['is_host_country'] = medal_standings_df['games_country'].eq(medal_standings_df['npc_name'])

In [5]:
print(medal_standings_df.shape)

medal_athlete_df = medal_athlete_df[medal_athlete_df["npc_new"] != "NA"]
medal_athlete_df = medal_athlete_df[medal_athlete_df["npc_new"] != np.nan]

print(medal_standings_df.shape)

(2346, 23)
(2346, 23)


In [6]:
medal_athlete_df["npc_name"].value_counts()

United States     3059
Germany           2334
United Kingdom    2174
Canada            1538
France            1519
                  ... 
Sudan                1
Liechtenstein        1
Cape Verde           1
Rwanda               1
Chile                1
Name: npc_name, Length: 115, dtype: int64

In [7]:
print(medal_standings_df.shape)
print(medal_athlete_df.shape)

(2346, 23)
(28906, 28)


In [8]:
medal_athlete_df["games_code-games_season-sport-npc"] = medal_athlete_df["games_code"] + "-" + medal_athlete_df["games_season"] + "-" + medal_athlete_df["sport"] + "-" + medal_athlete_df["npc"]
medal_standings_df["games_code-games_season-sport-npc"] = medal_standings_df["games_code"] + "-" + medal_standings_df["games_season"] + "-" + medal_standings_df["sport"]+ "-" + medal_standings_df["npc"]

In [9]:
medal_athlete_df.columns,medal_standings_df.columns

(Index(['games_code', 'games_year', 'games_city', 'City_latitude',
        'City_longitude', 'games_country', 'Country_latitude',
        'Country_longitude', 'games_continent', 'games_start', 'games_end',
        'games_season', 'sport', 'sport_code', 'event_venue', 'events', 'npcs',
        'athletes', 'event', 'medal', 'npc', 'npc_new', 'npc_name',
        'npc_name_latitude', 'npc_name_longitude', 'athlete_name',
        'athlete_info_og', 'medals_per_athlete',
        'games_code-games_season-sport-npc'],
       dtype='object'),
 Index(['games_code', 'games_year', 'games_city', 'games_country',
        'games_continent', 'games_start', 'games_end', 'games_season', 'npc',
        'npc_new', 'npc_name', 'rank_type', 'npc_rank', 'npc_gold',
        'npc_silver', 'npc_bronze', 'sport_code', 'sport', 'npc_total_medals',
        'games_duration', 'medal_density', 'is_top_performer',
        'is_host_country', 'games_code-games_season-sport-npc'],
       dtype='object'))

In [10]:
medal_athlete_df = medal_athlete_df[['games_code', 'games_season', 'sport', 'sport_code', 'event_venue', 'events', 'npcs',
        'athletes', 'event', 'medal', 'npc', 'npc_new', 'npc_name','npc_name_latitude', 'npc_name_longitude', 'athlete_name',
        'athlete_info_og', 'medals_per_athlete','games_code-games_season-sport-npc']]

In [11]:
# Save the cleaned datasets to Excel files
medal_standings_df.to_excel(r'C:\Users\U6079006\Desktop\PowerBI Projects\Onyx Project\September\cleaned_medal_standings_df.xlsx', index=False)
medal_athlete_df.to_excel(r'C:\Users\U6079006\Desktop\PowerBI Projects\Onyx Project\September\cleaned_medal_athlete_df.xlsx', index=False)